In [1]:
import gdown
file_id = "1cPpoB-xgnZQrKWgnvA3Wohpsyq_E8vyq"
file_name = "email_spam_indo.csv"
gdown.download(f"https://drive.google.com/uc?id={file_id}", file_name, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1cPpoB-xgnZQrKWgnvA3Wohpsyq_E8vyq
To: /content/email_spam_indo.csv
100%|██████████| 3.93M/3.93M [00:00<00:00, 19.9MB/s]


'email_spam_indo.csv'

In [2]:
import pandas as pd
df = pd.read_csv(file_name)
df.head()

,Kategori,Pesan
0,spam,Secara alami tak tertahankan identitas perusah...
1,spam,Fanny Gunslinger Perdagangan Saham adalah Merr...
2,spam,Rumah -rumah baru yang luar biasa menjadi muda...
3,spam,4 Permintaan Khusus Pencetakan Warna Informasi...
4,spam,"Jangan punya uang, dapatkan CD perangkat lunak..."


In [3]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

df["Pesan"] = df["Pesan"].astype(str).apply(clean_text)
stopwords_id = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words=list(stopwords_id))
features_vectorized = vectorizer.fit_transform(df["Pesan"].values)

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_vectorized, df["Kategori"].values, test_size=0.2, random_state=42)

In [6]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=5000)
clf.fit(x_train, y_train)

LogisticRegression(max_iter=5000)

In [7]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = clf.predict(x_test)
print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))

Akurasi: 0.9715909090909091

Laporan Klasifikasi:
               precision    recall  f1-score   support

         ham       1.00      0.94      0.97       263
        spam       0.95      1.00      0.97       265

    accuracy                           0.97       528
   macro avg       0.97      0.97      0.97       528
weighted avg       0.97      0.97      0.97       528



In [8]:
def prediksi_spam(teks):
    teks_bersih = clean_text(teks)
    teks_vector = vectorizer.transform([teks_bersih])
    hasil = clf.predict(teks_vector)
    return hasil[0]


In [9]:
# Contoh Penggunaan Model dengan Data dari Dataset
contoh_pesan = df["Pesan"].sample(5, random_state=42).tolist()
for i, pesan in enumerate(contoh_pesan):
    print(f"Pesan {i+1}: {pesan}")
    print("Klasifikasi:", prediksi_spam(pesan))
    print("-")

Pesan 1: wawancara telepon dengan enron research group good morning richard resume anda diteruskan ke vince kaminski dan kelompok riset dan mereka ingin melakukan wawancara telepon dengan anda sesuka anda tolong beri saya beberapa tanggal dan waktu anda akan tersedia dan saya akan mengoordinasikan jadwal juga nomor telepon yang ingin anda hubungi wawancara telepon akan berlangsung sekitar satu jam dan pewawancara adalah direktur pelaksana vince kaminski wakil presiden penelitian stinson gibner penelitian vasant shanbhogue wakil presiden penelitian terima kasih richard dan kami berharap dapat mendengar dari anda salam koordinator administrasi shirley crenshaw enron research group      email shirley crenshaw  enron com
Klasifikasi: ham
-
Pesan 2: re posisi magang musim panas hi vince paulo oleira salah satu m i t menghadiri pertemuan kami pada wed minat penelitian ternyata menjadi pertandingan untuk april hodgeson vp of content origination saya meminta dia berbicara dengan april stinson 